<a href="https://colab.research.google.com/github/ASR-X/Antiddiction/blob/master/Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
!wget https://www.datafiles.samhsa.gov/sites/default/files/field-uploads-protected/studies/TEDS-A-2018/TEDS-A-2018-datasets/TEDS-A-2018-DS0001/TEDS-A-2018-DS0001-bundles-with-study-info/TEDS-A-2018-DS0001-bndl-data-tsv.zip

--2021-08-03 14:54:05--  https://www.datafiles.samhsa.gov/sites/default/files/field-uploads-protected/studies/TEDS-A-2018/TEDS-A-2018-datasets/TEDS-A-2018-DS0001/TEDS-A-2018-DS0001-bundles-with-study-info/TEDS-A-2018-DS0001-bndl-data-tsv.zip
Resolving www.datafiles.samhsa.gov (www.datafiles.samhsa.gov)... 52.44.92.150, 54.164.153.73
Connecting to www.datafiles.samhsa.gov (www.datafiles.samhsa.gov)|52.44.92.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39539653 (38M) [application/zip]
Saving to: ‘TEDS-A-2018-DS0001-bndl-data-tsv.zip’

TEDS-A-2018-DS0001- 100%[===================>]  37.71M  10.7MB/s    in 3.5s    

2021-08-03 14:54:10 (10.7 MB/s) - ‘TEDS-A-2018-DS0001-bndl-data-tsv.zip’ saved [39539653/39539653]



In [ ]:
!unzip /content/TEDS-A-2018-DS0001-bndl-data-tsv.zip

Archive:  /content/TEDS-A-2018-DS0001-bndl-data-tsv.zip
  inflating: tedsa_puf_2018.csv      


In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('/content/tedsa_puf_2018.csv',header="infer")
x=np.array(df[["ALCDRUG", "ROUTE1","FRSTUSE1","EMPLOY","ALCFLG","AMPHFLG","BARBFLG","BENZFLG","COKEFLG","FREQ_ATND_SELF_HELP","GENDER","HALLFLG","HERFLG","HLTHINS","INHFLG","MARFLG","MARSTAT","MTHAMFLG","OTCFLG"]])
y=np.array(df["FREQ1"])

In [ ]:

testx=np.array(df.head(100000)[["ALCDRUG", "ROUTE1","FRSTUSE1","EMPLOY","ALCFLG","AMPHFLG","BARBFLG","BENZFLG","COKEFLG","FREQ_ATND_SELF_HELP","GENDER","HALLFLG","HERFLG","HLTHINS","INHFLG","MARFLG","MARSTAT","MTHAMFLG","OTCFLG"]])
testy=np.array(df.head(100000)["FREQ1"])

In [ ]:
from matplotlib.pyplot import figure

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
# plot decision tree
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
# fit model
model = XGBClassifier()
model.fit(x, y)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
import pickle 
pickle.dump(model, open("xgb_model.pkl", "wb"))


In [ ]:
# plot single tree
plot_tree(model,num_trees=5)
plt.show()
1-(np.count_nonzero(np.round(model.predict(testx)).flatten()-testy)/100000)

# New Section

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
import pickle
def load_pkl(fname):
    with open(fname, 'rb') as f:
        obj = pickle.load(f)
    return obj

model = load_pkl('xgb_model.pkl')

In [ ]:
%%capture
!pip install fastapi nest-asyncio pyngrok uvicorn python-multipart

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


!./ngrok authtoken 1rKW512YaaAic3Lmwx2ZKtyfs1i_79mpcmRvUwQEVb4GTAeCP
!apt-get install screen


In [ ]:
!screen -dm bash -c 'ngrok http -region=us -hostname=asrx.ngrok.io 8000'

In [ ]:

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel
import asyncio
from time import sleep
import re
from typing import List
import numpy as np
from datetime import datetime, timedelta
import json
app = FastAPI()


app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)
class SurveyIn(BaseModel):
        age: str
        firstAge: str
        gender:  str
        marital:  str
        education:  str
        employment:  str
        insurance:  str
        drugs: List[str] =[]
        admin:  str
        dose: List[str] =[]
    

class OutModel(BaseModel):
      riskLevel: str
      probs: List[str] =[]
      timestamps: List[str] =[]
    

def date_range( step=1, date_format="%Y-%m-%d"):
    start = datetime.today()
    end = start + timedelta(days=100)
    num_days = (end - start).days
    
    for d in range(0, num_days + step, step):
        date_i = start + timedelta(days=d)
        yield date_i.strftime(date_format)

def getTimeline(stage, history):
  def f(x):
    x=x+np.random.normal(loc=0, scale=0.1, size=None)
    return 1*np.log2(x+1)/(x+7)+np.random.normal(loc=0, scale=np.abs(0.1*np.log2(x+1)/(x+7)), size=None)
  input=history
  if not len(input)<3:
    diffs=[j-i for i, j in zip(input[:-1], input[1:])]
    avg=sum(diffs) / len(diffs)
  else:
    avg=30
  if avg<3:x=np.linspace(5,105,100)
  elif avg>7 or stage!=1 :x=np.linspace(0+avg,100+avg,100)
  else:x=np.linspace(0,100,100)
  y=f(x)
  return y*(0.4 if stage==1 else (0.5 if stage==2 else 2))

def processInput(f):#f=factors
  a=np.array([int(d) for d in f["drugs"]])
  b = np.zeros((19))
  b[a]=1
  fi=int(f["firstAge"])
  cats=[12,15,18,21,25,30,35,40,45,50,55,60,65]
  firstAge=0
  for i in cats[::-1]:
    if fi>=i:
      firstAge=cats.index(i)+1
      break
  
  return [0,
        f["admin"],
        firstAge,
        f["employment"],
        b[2],
        b[11],
        b[15],
        b[7],
        b[8],
        0,
        f["gender"],
        b[9],
        b[5],
        f["insurance"],
        b[17],
        b[4],
        f["marital"],
        b[10],
        b[18],]

@app.api_route("/", methods=["GET", "POST"])
async def root():
    return 'n/a'
    
@app.post('/getProb', response_model=OutModel)
async def ask(answers: SurveyIn):
  answers=json.loads(answers.json())

  history=answers["dose"]
  g=model.predict([int(x) for x in processInput(answers)])
  g=1 if g==-9 else g
  dates=[datetime.strptime(str(x), "%Y-%m-%d").timestamp()/24/3600 for x in history]
  dates.sort()
  predProbs=getTimeline(g,dates)
  timelines=[d for d in date_range()]
  for i in range(len(timelines)):
    if i%10!=0:
      timelines[i]=""
  return {
      "riskLevel": str(g[0]),
      "probs": [str(d) for d in predProbs],
      "timestamps": timelines
    }

import nest_asyncio
from pyngrok import ngrok
import uvicorn
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [75]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     47.187.210.143:0 - "POST /getProb HTTP/1.1" 200 OK
INFO:     47.187.210.143:0 - "POST /getProb HTTP/1.1" 200 OK


Streaming output truncated to the last 5000 lines.
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7efe5d6e8be0> is already entered
Exception in callback BaseAsyncIOLoop._handle_events(17, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(17, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7efe5d6e8be0> is already entered
Exception in callback BaseAsyncIOLoop._handle_events(17, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(17, 1)>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <Context object at 0x7efe5d6e8be0> is already entered
Exception in callback BaseAsyncIOLoop._handle_events(17, 1)
handle: <Handle BaseAsyncIOLoop._handl